# Wikipics demo

In [1]:
from pathlib import Path
from typing import NamedTuple
from ipywidgets.widgets import Image, Layout, HBox

import httpx

from wikipics import get_sample_url, get_sample_urls

class ImageRecord(NamedTuple):
    pixels: bytes
    name: str
    size: int

In [2]:
def fetch(url) -> ImageRecord:
    resp = httpx.get(url)
    resp.raise_for_status()
    name = Path(url).name
    return ImageRecord(resp.content, name, len(resp.content))

In [3]:
#img_rec = fetch(get_sample_url(1_000_000))
#Image(value=img_rec.pixels)

In [4]:
%%time
img_widgets = []
qty = 10
scale = f'{100//qty}%'
urls = get_sample_urls(2_000_000, qty)
total_bytes = 0
for url in urls:
    img_rec = fetch(url)
    print(f'{img_rec.size:12_} bytes | {img_rec.name}')
    total_bytes += img_rec.size
    img_widgets.append(Image(value=img_rec.pixels, layout=Layout(width=scale, height=scale)))

print(f'TOTAL BYTES: {total_bytes:_}')
HBox(img_widgets)

   1_968_070 bytes | Io_highest_resolution_true_color.jpg
   1_976_293 bytes | United_States_Declaration_of_Independence.jpg
   1_996_200 bytes | Bison_skull_pile_edit.jpg
   2_019_740 bytes | Bald.eagle.closeup.arp-sh.750pix.jpg
   1_985_830 bytes | Plains_Zebra_Equus_quagga.jpg
   1_974_653 bytes | Phascolarctos_cinereus_Bonorong.jpg
   2_020_431 bytes | Kaiseradler_Aquila_heliaca_2_amk.jpg
   1_993_394 bytes | Peacock_butterfly_%28Aglais_io%29_2.jpg
   2_010_689 bytes | Morocco_Africa_Flickr_Rosino_December_2005_84514010.jpg
   2_004_178 bytes | Thiodina_puerpera_female_02.jpg
TOTAL BYTES: 19_949_478
CPU times: user 758 ms, sys: 276 ms, total: 1.03 s
Wall time: 16.5 s


In [5]:
class Gallery:
    def __init__(self, num_images):
        scale = f'{100//num_images}%'
        self.img_layout = Layout(width=scale, height=scale)
        no_img = open('no-image.png', 'rb').read()
        self.img_widgets = [Image(value=no_img, layout=self.img_layout)] * num_images
        self.box = HBox(self.img_widgets)
        self.size = 0

    def display(self):
        display(self.box)

    def update(self, index, pixels):
        self.size += len(pixels)
        self.img_widgets[index] = Image(value=pixels, alt=f'({index})', layout=self.img_layout)
        self.box.children = self.img_widgets
        print(f'({index+1:2}){img_rec.size:12_} bytes | {img_rec.name}', flush=True)


In [6]:
%%time
num_images = 10
gallery = Gallery(num_images)
gallery.display()

urls = get_sample_urls(2_000_000, num_images)

for i, url in enumerate(urls):
    img_rec = fetch(url)
    total_bytes += img_rec.size
    gallery.update(i, img_rec.pixels)

print(f'TOTAL BYTES: {gallery.size:_}')

( 1)   1_992_141 bytes | Thalassarche_cauta_-_SE_Tasmania.jpg
( 2)   2_015_977 bytes | Litoria_infrafrenata_-_Julatten.jpg
( 3)   2_006_701 bytes | Dhow_znz.jpg
( 4)   1_976_553 bytes | Photographing_a_model.jpg
( 5)   2_010_689 bytes | Morocco_Africa_Flickr_Rosino_December_2005_84514010.jpg
( 6)   2_010_325 bytes | Rana_esculenta_on_Nymphaea_edit.JPG
( 7)   2_019_740 bytes | Bald.eagle.closeup.arp-sh.750pix.jpg
( 8)   1_992_966 bytes | Aeshna_cyanea_freshly_slipped_with_time.jpg
( 9)   2_020_431 bytes | Kaiseradler_Aquila_heliaca_2_amk.jpg
(10)   1_975_417 bytes | Dolceacqua43_-_Artista_locale_mentre_dipinge_un_acquarello.jpg
TOTAL BYTES: 20_020_940
CPU times: user 760 ms, sys: 261 ms, total: 1.02 s
Wall time: 16.5 s


In [7]:
%%time
from concurrent import futures

num_images = 20
gallery = Gallery(num_images)
gallery.display()

urls = get_sample_urls(2_000_000, num_images)
with futures.ThreadPoolExecutor() as pool:
    img_records = pool.map(fetch, urls)
    for i, img_rec in enumerate(img_records):
        gallery.update(i, img_rec.pixels)

print(f'TOTAL BYTES: {gallery.size:_}')

( 1)   2_044_892 bytes | Dacelo_novaeguineae_waterworks.jpg
( 2)   1_992_141 bytes | Thalassarche_cauta_-_SE_Tasmania.jpg
( 3)   2_017_570 bytes | Bharata_Natyam_Performance_DS.jpg
( 4)   1_976_553 bytes | Photographing_a_model.jpg
( 5)   1_963_394 bytes | Synchiropus_splendidus_2_Luc_Viatour.jpg
( 6)   2_035_926 bytes | Agnes_Milowka_by_J_Axford.jpg
( 7)   2_025_198 bytes | Mansudae-Monument-Bow-2014.jpg
( 8)   1_937_954 bytes | Pholiota_malicola.jpg
( 9)   2_036_802 bytes | Coenagrionidae2.jpg
(10)   2_006_701 bytes | Dhow_znz.jpg
(11)   1_163_231 bytes | Portrait_of_Jupiter_from_Cassini.jpg
(12)   2_036_964 bytes | Champ_de_Mars_from_the_Eiffel_Tower_-_July_2006_edit.jpg
(13)   2_041_482 bytes | Pippin-Atmark-Console-Set.jpg
(14)   2_029_273 bytes | Chandiroor_Divakaran_New_DSW.JPG
(15)   1_968_070 bytes | Io_highest_resolution_true_color.jpg
(16)   1_962_254 bytes | Inside_the_Tarawera_rift.jpg
(17)   2_015_977 bytes | Litoria_infrafrenata_-_Julatten.jpg
(18)   2_019_740 bytes | Ba

In [8]:
%%time
from IPython.display import display
from concurrent import futures

num_images = 20
gallery = Gallery(num_images)
gallery.display()
urls = get_sample_urls(2_000_000, num_images)

with futures.ThreadPoolExecutor() as pool:
    tasks = [pool.submit(fetch, url) for url in urls]
    for i, future in enumerate(futures.as_completed(tasks)):
        img_rec = future.result()
        gallery.update(i, img_rec.pixels)
        
print(f'TOTAL BYTES: {gallery.size:_}')

( 1)   1_984_907 bytes | Glasshouse_and_fountain_at_lalbagh.jpg
( 2)   2_029_322 bytes | Albert_Bierstadt_-_The_Rocky_Mountains%2C_Lander%27s_Peak.jpg
( 3)   1_974_272 bytes | Panorpa_alpina_2_Luc_Viatour.jpg
( 4)   2_041_623 bytes | Setophaga_coronata_MP.jpg
( 5)   1_992_141 bytes | Thalassarche_cauta_-_SE_Tasmania.jpg
( 6)   2_041_482 bytes | Pippin-Atmark-Console-Set.jpg
( 7)   1_994_699 bytes | BennyTrapp_Hyla_intermedia_Italien.jpg
( 8)   1_940_579 bytes | Amundsen-Scott_marsstation_ray_h_edit.jpg
( 9)   2_019_580 bytes | Edvard_Munch_-_The_Kiss_-_Google_Art_Project.jpg
(10)   1_962_254 bytes | Inside_the_Tarawera_rift.jpg
(11)   2_040_629 bytes | Sony_A77_II.jpg
(12)   2_752_180 bytes | F-16_June_2008.jpg
(13)   2_004_178 bytes | Thiodina_puerpera_female_02.jpg
(14)   2_015_977 bytes | Litoria_infrafrenata_-_Julatten.jpg
(15)   1_943_747 bytes | Van_dyck-el_pintor_martin_ryckaert-prado.jpg
(16)   2_036_964 bytes | Champ_de_Mars_from_the_Eiffel_Tower_-_July_2006_edit.jpg
(17)   1_